# Description

Описание задачи
Ваша компания «переезжает» с упрощенной системы налогообложения на общую. К вам подошел главный бухгалтер с просьбой:

В нашей сети около 100 аптек. Нам нужно единоразово загрузить файл в 1С, но для этого надо сделать сравнение реализации по аптекам и выгрузки из СБИСа. Пункты, которые расходятся, мы подправим вручную. После этого будем загружать в 1С. Подробная информация ниже.

Выгрузка из СБИС находится в папке Входящие. Загрузите все csv файлы в один датафрейм. Если встретился файл с другим расширением, то его пропускаем. Будьте аккуратны с заголовками столбцов - не мешайте их с данными при объединении нескольких файлов.

Дайте столбцам полученного датафрейма такие названия:

"Дата",
"Номер",
"Сумма",
"Статус",
"Примечание",
"Комментарий",
"Контрагент",
"ИНН/КПП",
"Организация",
"ИНН/КПП",
"Тип документа",
"Имя файла",
"Дата",
"Номер 1",
"Сумма 1",
"Сумма НДС",
"Ответственный",
"Подразделение",
"Код",
"Дата",
"Время",
"Тип пакета",
"Идентификатор пакета",
"Запущено в обработку",
"Получено контрагентом",
"Завершено",
"Увеличение суммы",
"НДC",
"Уменьшение суммы",
"НДС"

Все названия из нескольких слов сцепите через нижнее подчеркивание в одно_длинное_слово.
Файлы с выгрузкой из аптек находятся в папке Аптеки/csv/correct/.
Загружайте на обработку по одному файлу. Содержимое каждого файла загружайте в датафрейм Pandas. Если встречаете не csv формат - игнорируйте его.
Создайте столбцы:

"Номер счет-фактуры"
"Сумма счет-фактуры"
"Дата счет-фактуры"
"Сравнение дат"

В каждой строке проверьте:
Если "Поставщик" - ЕАПТЕКА, то к "Номер накладной" нужно добавить /15.
Нужно найти все записи в выгрузке из СБИСа по данному номеру накладной
Из найденных строк нужно оставить только те, которые имеют один из типов документа: ["СчФктр", "УпдДоп", "УпдСчфДоп", "ЭДОНакл"]
Если ничего не найдено - просто переходим к следующей строке
Если найдено - нужно сохранить значения Номер, Сумма и Дата
Дату нужно представить в формате 25.05.2021
В столбцы из пункта 6 нужно записать найденные для данной строки значения
В столбец Сравнение дат помещаем "Не совпадает!", если найденная дата и дата накладной отличаются. Иначе - пустая строка.
Итоговый файл выгрузки по аптеке должен содержать такие столбцы в заданном порядке:
'№ п/п', 'Штрих-код партии', 'Наименование товара', 'Поставщик',
'Дата приходного документа', 'Номер приходного документа',
'Дата накладной', 'Номер накладной', 'Номер счет-фактуры',
'Сумма счет-фактуры', 'Кол-во',
'Сумма в закупочных ценах без НДС', 'Ставка НДС поставщика',
'Сумма НДС', 'Сумма в закупочных ценах с НДС', 'Дата счет-фактуры', 'Сравнение дат'
Файл сохраните по пути Результат/{полная сегодняшняя дата}/{имя исходного файла без расширения} - результат.xlsx. 
Если таких папок не существует - создайте их (проверку и создание нужно реализовать средствами Python, автоматически).

In [3]:
import os
from datetime import datetime

import pandas as pd

result_path = f'Результат/{datetime.now().strftime("%Y-%m-%d")}/'
if not os.path.exists(result_path):
    os.makedirs(result_path)

sbis_names = ["Дата",
            "Номер",
            "Сумма",
            "Статус",
            "Примечание",
            "Комментарий",
            "Контрагент",
            "ИНН/КПП",
            "Организация",
            "ИНН/КПП",
            "Тип документа",
            "Имя файла",
            "Дата",
            "Номер 1",
            "Сумма 1",
            "Сумма НДС",
            "Ответственный",
            "Подразделение",
            "Код",
            "Дата",
            "Время",
            "Тип пакета",
            "Идентификатор пакета",
            "Запущено в обработку",
            "Получено контрагентом",
            "Завершено",
            "Увеличение суммы",
            "НДC",
            "Уменьшение суммы",
            "НДС"
            ]

sbis_folder = "Входящие"
sbis_files = os.listdir(sbis_folder)

dfs = []
for file in sbis_files:
    if 'csv' not in file:
        continue
    df = pd.read_csv(sbis_folder + "/" + file, skiprows=1, sep = ";", encoding="windows-1251", header=None)
    dfs.append(df)

sbis = pd.concat(dfs, ignore_index=True)
sbis.columns = sbis_names
sbis.columns = [c.replace(' ', '_') for c in sbis.columns]

apteka_folder = "Аптеки/csv/correct/"
apteka_files = os.listdir(apteka_folder)

for file in apteka_files:

    if 'csv' not in file:
        continue

    apteka = pd.read_csv(apteka_folder + file, sep = ";", encoding="windows-1251")
    apteka["Номер счет-фактуры"] = ""
    apteka["Сумма счет-фактуры"] = ""
    apteka["Дата счет-фактуры"] = ""
    apteka["Сравнение дат"] = ""

    docs = ["СчФктр", "УпдДоп", "УпдСчфДоп", "ЭДОНакл"]

    for i, row in apteka.iterrows():
        nakl = row["Номер накладной"]

        if 'ЕАПТЕКА' in row["Поставщик"]:
            nakl += "/15"

        records = sbis[sbis.Номер.values == nakl]
        records = records[records.Тип_документа.isin(docs)]

        if records.empty:
            continue

        invoice = records.iloc[0]["Номер"]
        summ = records.iloc[0]["Сумма"]
        date = records.iloc[0]["Дата"][1]
        date = datetime.strptime(date, "%d.%m.%y").strftime("%d.%m.%Y")

        apteka.at[i, "Номер счет-фактуры"] = invoice
        apteka.at[i, "Сумма счет-фактуры"] = summ
        apteka.at[i, "Дата счет-фактуры"] = date
        apteka.at[i, "Сравнение дат"] = "" if (date == apteka.at[i, 'Дата накладной']) else "Не совпадает!"


    apteka_columns = ['№ п/п', 'Штрих-код партии', 'Наименование товара', 'Поставщик',
        'Дата приходного документа', 'Номер приходного документа',
        'Дата накладной', 'Номер накладной', 'Номер счет-фактуры',
        'Сумма счет-фактуры', 'Кол-во',
        'Сумма в закупочных ценах без НДС', 'Ставка НДС поставщика',
        'Сумма НДС', 'Сумма в закупочных ценах с НДС', 'Дата счет-фактуры', 'Сравнение дат']

    apteka = apteka[apteka_columns]
    apteka.to_excel(f"{result_path}{file.split('.csv')[0]} - результат.xlsx", index=False)
    print(f'{file} Обработан!')


/tmp/ipykernel_852827/3518189988.py:86: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  date = records.iloc[0]["Дата"][1]


366.csv Обработан!


/tmp/ipykernel_852827/3518189988.py:86: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  date = records.iloc[0]["Дата"][1]


А123.csv Обработан!
